In [1]:
from bs4 import BeautifulSoup
import re
import codecs
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import pymorphy2 
from pymystem3 import Mystem

In [2]:
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

### mystem

In [3]:
output_txt = 'h1_h6_output_mystem.txt'
total_sites = 28026
path = '../content/'

In [4]:
m = Mystem()

with tqdm_notebook(total=total_sites) as pbar:
    with open(output_txt, 'w') as out_f:
        out_f.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format('doc_id', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'))
        for doc_id in range(1,total_sites+1):
            out_f.write('{}\t'.format(doc_id))
            with codecs.open(path + str(doc_id) + '.dat', 'r', 'utf-8') as in_f:
                soup = BeautifulSoup(in_f, 'lxml')
                for i in range(1,7):
                    h_all = soup.find_all('h{}'.format(i))
                    result = [i for i in re.split(r'\W+', str(h_all).lower()) if i]
                    result = [re.sub(r'[^ёЁА-я]', '', i) for i in result]
                    result = m.lemmatize(' '.join(result))
                    result = [u for u in result if len(u) > 2 and u not in russian_stopwords]
                    result = re.sub('\s+',' ', ' '.join(result))   
                        
                    if i == 6:
                        out_f.write('{}\n'.format(result))
                    else:
                        out_f.write('{}\t'.format(result)) 
                pbar.update()

In [5]:
df = pd.read_csv(output_txt, sep='\t', encoding='utf-8', lineterminator='\n')
df.head()

,doc_id,h1,h2,h3,h4,h5,h6
0,1,аншин центр репродукция генетика фертимед москва,температура тело особенность поведение особенн...,NaN,NaN,NaN,NaN
1,2,перевод киви кошелек,переводить деньги киви кошелек киви кошелек п...,навигация сайт,NaN,NaN,NaN
2,3,проект патруль время реабилитация духовный сущ...,деньги экономика обмен финансы деньги процент,деньги экономика обмен финансы деньги процент...,NaN,NaN,NaN
3,4,клуб преподавание начальный класс блог лента ...,NaN,NaN,NaN,NaN,NaN
4,5,быстро понижать холестерин высокий холестерин,норма холестерин кровь правильно питаться сниж...,липа снижение холестерин фасоль снижать холест...,комментарий обновлять список комментарий,NaN,NaN


In [6]:
df[['doc_id', 'h1']].to_csv('h1_mystem.txt',  index=False, sep='\t')
df[['doc_id', 'h2']].to_csv('h2_mystem.txt',  index=False, sep='\t')
df[['doc_id', 'h3']].to_csv('h3_mystem.txt',  index=False, sep='\t')
df[['doc_id', 'h4']].to_csv('h4_mystem.txt',  index=False, sep='\t')
df[['doc_id', 'h5']].to_csv('h5_mystem.txt',  index=False, sep='\t')
df[['doc_id', 'h6']].to_csv('h6_mystem.txt',  index=False, sep='\t')

### pymorphy2

In [3]:
output_txt = 'h1_h6_output_pymorphy2_tail2.txt'
total_sites = 28026
path = '../content/'

In [4]:
def pos(word, morth=pymorphy2.MorphAnalyzer()):
    "Return a likely part of speech for the *word*."""
    return morth.parse(word)[0].tag.POS
functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}  # function words

In [5]:
doc_to_title = {}
with tqdm_notebook(total=total_sites) as pbar:
    with open(output_txt, 'w') as out_f:
        out_f.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format('doc_id', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'))
        for doc_id in range(1,total_sites+1):
            out_f.write('{}\t'.format(doc_id))
            with open(path + str(doc_id) + ".dat",'r') as in_f:
                soup = BeautifulSoup(in_f, 'html.parser')
                for i in range(1,7):   
                    h_all = str(soup.find_all('h{}'.format(i))).lower()
                    result = [i for i in re.split(r'\W+', h_all) if i] 
                    result = [re.sub(r'[^ёЁА-я]', '', i) for i in result]
                    morth=pymorphy2.MorphAnalyzer()
                    result = [morth.parse(word)[0].normal_form  for word in result if \
                              pos(word) not in functors_pos]
                    result = [word for word in result if word not in russian_stopwords and len(word)>2]
                    result = re.sub('\s+',' ', ' '.join(result))   

                    if i == 6:
                        out_f.write('{}\n'.format(result))
                    else:
                        out_f.write('{}\t'.format(result)) 
                pbar.update()

In [6]:
df = pd.read_csv(output_txt, sep='\t', encoding='utf-8', lineterminator='\n')
df.head()

,doc_id,h1,h2,h3,h4,h5,h6
0,1,метр аншина центр репродукция генетик фертимед...,температура тело особенность поведение особенн...,NaN,NaN,NaN,NaN
1,2,перевод киви кошелёк,перевести деньга киви кошелёк киви кошелёк пер...,навигация сайт,NaN,NaN,NaN
2,3,проект патруль время реабилитация духовный сущ...,деньга экономика обмен финансы деньга процент,деньга экономика обмен финансы деньга процент ...,NaN,NaN,NaN
3,4,клуб преподавание начальный класс блог лента л...,NaN,NaN,NaN,NaN,NaN
4,5,быстро понизить холестерин высокий холестерин,норма холестерин кровь правильно питаться сниз...,липа снижение холестерин фасоль снизить холест...,комментарий обновить список комментарий,NaN,NaN


In [7]:
df[['doc_id', 'h1']].to_csv('h1_pymorphy2.txt',  index=False, sep='\t')
df[['doc_id', 'h2']].to_csv('h2_pymorphy2.txt',  index=False, sep='\t')
df[['doc_id', 'h3']].to_csv('h3_pymorphy2.txt',  index=False, sep='\t')
df[['doc_id', 'h4']].to_csv('h4_pymorphy2.txt',  index=False, sep='\t')
df[['doc_id', 'h5']].to_csv('h5_pymorphy2.txt',  index=False, sep='\t')
df[['doc_id', 'h6']].to_csv('h6_pymorphy2.txt',  index=False, sep='\t')